In [32]:
import pandas as pd
import datetime

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [33]:
in_file_path = '../data/raw_2023_04_19/combined_data.csv'
out_file_path = '../data/raw_2023_04_19/clean_combined_data.csv'

In [34]:
import sys
sys.path.append(in_file_path)


data = pd.read_csv(in_file_path, header=None, on_bad_lines='skip')
# data = data.dropna()


In [35]:

data.columns = column_names = [
"address",
"price",
"gross_tax",
"strata_maintenance_fees",
"bedrooms",
"bathrooms",
"property_type",
"property_age",
"title",
"style",
"heating_type",
"feature",
"amenities",
"appliances",
"community",
"days_on_rew",
"property_views",
"mls®_number",
"source",
'frontage', 
'lot_size', 
'year_built', 
'depth',
'half_bathrooms'
]
data.head()


,address,price,gross_tax,strata_maintenance_fees,bedrooms,bathrooms,property_type,property_age,title,style,...,community,days_on_rew,property_views,mls®_number,source,frontage,lot_size,year_built,depth,half_bathrooms
0,1106 Grover Avenue,"$3,488,000","$4,504",NaN,7.0,7.0,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,central coquitlam,8 days,642.0,r2766859,rebgv,63.00 feet,63 ft x 125 ft (7875 ft²),built in 2023 (0 yrs old),125,2.0
1,213 617 Smith Avenue,"$549,000","$1,593",$272,1.0,1.0,apt/condo,NaN,freehold strata,inside unit,...,coquitlam west,13 days,910.0,r2766119,rebgv,NaN,NaN,built in 2013 (10 yrs old),NaN,0.0
2,1241 Oxbow Way,"$1,100,000","$3,316",$135,3.0,3.0,house,NaN,freehold strata,2 storey,...,river springs,1 day,NaN,r2769641,rebgv,0.00 feet,4104 ft²,built in 1980 (43 yrs old),NaN,0.0
3,2 271 Blue Mountain Street,"$1,170,000",$0,$0,3.0,2.0,duplex,NaN,freehold strata,2 storey,...,coquitlam west,1 day,NaN,r2769587,rebgv,NaN,0 ft²,built in 2023 (0 yrs old),NaN,1.0
4,315 3176 Plateau Boulevard,"$675,000","$1,785",$447,2.0,2.0,apt/condo,NaN,freehold strata,"corner unit,upper unit",...,westwood plateau,1 day,NaN,r2768108,rebgv,NaN,NaN,built in 1997 (26 yrs old),NaN,0.0


In [36]:
# Filter for Houses Only
houses = data[data['property_type'] == 'house']

In [37]:
# Convert Price column to integer
houses["price"] = houses["price"].str.replace(",", "").str.replace("$", "")
houses["price"] = pd.to_numeric(houses["price"])

In [38]:
# extract the lot width from the lot_size column
houses["lot_width"] = houses["lot_size"].str.extract(r"^(\d+) ft x")

# extract the lot length from the lot_size column
houses["lot_length"] = houses["lot_size"].str.extract(r"(\d+) ft x")

# extract the lot size from the lot_size column
houses["lot_size"] = houses["lot_size"].str.extract(r"\((\d+) ft²\)")


In [39]:
# Convert Gross Tax Income to Integer
houses['gross_tax'] = houses['gross_tax'].fillna(0)
houses['gross_tax'] = houses['gross_tax'].astype(str)
houses['gross_tax'] = houses['gross_tax'].str.replace(',', '')
houses['gross_tax'] = houses['gross_tax'].str.replace('$', '')
houses['gross_tax'] = houses['gross_tax'].astype(int)

In [40]:
# Convert House Age to integer - Consider prebuild houses with age 0
# extract the year from the year_built column
houses["year_built"] = houses["year_built"].str.extract(r"(\d+)")

# compute the age of the house using the current year
current_year = datetime.datetime.now().year
houses["age"] = current_year - pd.to_numeric(houses["year_built"])

## remove duplicate based on mls number
houses = houses.drop_duplicates(subset=['mls®_number'])

In [41]:
# extract bungalow information
houses["bungalow"] = houses["style"].str.contains("bungalow").fillna(-1).astype(int)

# extract storey information
houses["storey"] = houses["style"].str.extract(r"(\d+)").fillna(-1).astype(int)

# extract basement information
houses["basement"] = houses["style"].str.contains(" w/bsmt").fillna(-1).astype(int)

# extract laneway house information
houses["laneway_house"] = houses["style"].str.contains("laneway house").fillna(-1).astype(int)

# Extract garage house information:
houses["garage"] = houses["feature"].str.contains("garage").fillna(-1).astype(int)


# extract split entry information
houses["split_entry"] = houses["style"].str.contains("split entry").fillna(-1).astype(int)
houses = houses.drop(['style'], axis=1)

In [42]:
# houses = houses.loc[:, ['address', 'price', 'gross_tax', 'bedrooms', 'bathrooms',
#                'property_type', 'community', 'lot_size', 'year_built', 'lot_width',
#                'lot_length', 'age', 'bungalow', 'storey', 'basement', 'laneway_house',
#                'garage', 'split_entry']]
houses.columns

Index(['address', 'price', 'gross_tax', 'strata_maintenance_fees', 'bedrooms',
       'bathrooms', 'property_type', 'property_age', 'title', 'heating_type',
       'feature', 'amenities', 'appliances', 'community', 'days_on_rew',
       'property_views', 'mls®_number', 'source', 'frontage', 'lot_size',
       'year_built', 'depth', 'half_bathrooms', 'lot_width', 'lot_length',
       'age', 'bungalow', 'storey', 'basement', 'laneway_house', 'garage',
       'split_entry'],
      dtype='object')

In [43]:
houses.head()


,address,price,gross_tax,strata_maintenance_fees,bedrooms,bathrooms,property_type,property_age,title,heating_type,...,half_bathrooms,lot_width,lot_length,age,bungalow,storey,basement,laneway_house,garage,split_entry
0,1106 Grover Avenue,3488000,4504,NaN,7.0,7.0,house,NaN,freehold nonstrata,heat pump,...,2.0,63,63,0.0,0,2,1,0,1,0
2,1241 Oxbow Way,1100000,3316,$135,3.0,3.0,house,NaN,freehold strata,"forced air,natural gas",...,0.0,NaN,NaN,43.0,0,2,0,0,-1,0
6,1414 Gabriola Drive,1450000,3867,NaN,4.0,2.0,house,NaN,freehold nonstrata,"forced air,heat pump",...,0.0,NaN,NaN,41.0,0,2,0,0,0,1
8,3200 Toba Drive,1379000,3789,NaN,4.0,2.0,house,NaN,freehold nonstrata,"forced air,natural gas",...,1.0,NaN,NaN,42.0,0,2,0,0,0,0
9,1709 Regan Avenue,2099900,5129,NaN,6.0,2.0,house,NaN,freehold nonstrata,forced air,...,0.0,72,72,64.0,0,2,0,0,-1,0


In [44]:
houses.to_csv(out_file_path, index=False)